In [2]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection  import train_test_split

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [5]:
print(train.head())
print(test.head())
print(submission.head())

   label  pixel0  pixel1  pixel2  ...  pixel780  pixel781  pixel782  pixel783
0      1       0       0       0  ...         0         0         0         0
1      0       0       0       0  ...         0         0         0         0
2      1       0       0       0  ...         0         0         0         0
3      4       0       0       0  ...         0         0         0         0
4      0       0       0       0  ...         0         0         0         0

[5 rows x 785 columns]
   pixel0  pixel1  pixel2  pixel3  ...  pixel780  pixel781  pixel782  pixel783
0       0       0       0       0  ...         0         0         0         0
1       0       0       0       0  ...         0         0         0         0
2       0       0       0       0  ...         0         0         0         0
3       0       0       0       0  ...         0         0         0         0
4       0       0       0       0  ...         0         0         0         0

[5 rows x 784 columns]
   ImageId

In [6]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [7]:
train = train.astype(float)
test = test.astype(float)

In [8]:
x = train.drop(columns = ['label'])
y = train.iloc[:,0]

In [10]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=14)

In [11]:
x_train = x_train/255.0
x_test = x_test/255.0
test = test/255.0

In [12]:
x_train = x_train.values.reshape(-1,28,28,1)
x_test = x_test.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [13]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [14]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu',input_shape=(28,28,1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1024, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              3277824   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [16]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
history = model.fit(x_train,y_train ,epochs = 50,validation_data=(x_test,y_test))

Epoch 1/50
1050/1050 [==============================] - 12s 5ms/step - loss: 0.2925 - accuracy: 0.9046 - val_loss: 0.0518 - val_accuracy: 0.9827
Epoch 2/50
1050/1050 [==============================] - 4s 4ms/step - loss: 0.0407 - accuracy: 0.9880 - val_loss: 0.0702 - val_accuracy: 0.9788
Epoch 3/50
1050/1050 [==============================] - 5s 4ms/step - loss: 0.0292 - accuracy: 0.9916 - val_loss: 0.0803 - val_accuracy: 0.9823
Epoch 4/50
1050/1050 [==============================] - 5s 4ms/step - loss: 0.0208 - accuracy: 0.9940 - val_loss: 0.0696 - val_accuracy: 0.9832
Epoch 5/50
1050/1050 [==============================] - 5s 4ms/step - loss: 0.0202 - accuracy: 0.9952 - val_loss: 0.0683 - val_accuracy: 0.9865
Epoch 6/50
1050/1050 [==============================] - 5s 4ms/step - loss: 0.0146 - accuracy: 0.9963 - val_loss: 0.0738 - val_accuracy: 0.9823
Epoch 7/50
1050/1050 [==============================] - 5s 4ms/step - loss: 0.0129 - accuracy: 0.9967 - val_loss: 0.0652 - val_accuracy

In [18]:
Y_pred = model.predict(test)

In [19]:
Y_pred

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [21]:
Y_pred=[np.argmax(i) for i in Y_pred]

In [22]:
Predictions = pd.DataFrame({'ImageId' : (i+1 for i in range(len(Y_pred))) , 'Label' : Y_pred})

In [23]:
Predictions.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [24]:
Submission_1 = Predictions.to_csv('Submission_1.csv',index=False)